In [ ]:
# default_exp pynamodbAttributes

# pynamodbAttributes
> special pynamodb attributes

In [ ]:
#export
from pynamodb.attributes import Attribute
import pynamodb

In [ ]:
#export
from typing import Any, Optional, Type, TypeVar
from enum import Enum
import requests, dpath.util, yaml, jsonschema, json, os

  
class SchemaAttribute(Attribute):
  attr_type = pynamodb.constants.STRING
  def __init__(self, schemaUrl:str, path:str = '/', isYaml=True, 
               headers={'Cache-Control': 'no-cache'}, 
               envName = 'SCHEMA_ATTRIBUTE', **kwargs: Any) -> None:
      """
      schemaUrl:str, 
      path:str = '/', 
      isYaml=True,  :yaml::Bool:: whether the schema is in yaml or json
      headers={'Cache-Control': 'no-cache'},
      :path::str:: the path of the object of interest in schema, if the schema is at root then '/'
      envName::str:: the name of schema to save to the environment
      """
      super().__init__(**kwargs)
      try:
        if isYaml: # yaml schema
          schema:dict = yaml.load(requests.get(schemaUrl, headers=headers).text, Loader = yaml.FullLoader)
        else: # probably json
          schema:dict = requests.get(schemaUrl, headers).json()
      except Exception as e:
        print(f'error parsing schema {e}')
        schema:dict = {}
          
      self.schema = dpath.util.get(schema, path) # get to the path in schema
      os.environ[envName] = json.dumps(self.schema)

  def deserialize(self, value: str) -> dict:
    return json.loads(value)

  def serialize(self, value:dict) -> str:
    res = jsonschema.validate(value,self.schema)
    return json.dumps(value)

### Test

In [ ]:
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute
import json

schemaUrl = 'https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/master/Product.json'

class TestModel(Model):
  class Meta:
    table_name="colab-test-sensitive-column"
    region = 'ap-southeast-1'
  phoneHash = UnicodeAttribute(hash_key=True)
  stringData = UnicodeAttribute()
  schemaThing = SchemaAttribute(schemaUrl = schemaUrl, null=True)
#   data =
  def __repr__(self):
    return json.dumps(vars(self)['attribute_values'])
  


### success

In [ ]:
try:
  TestModel(
    phoneHash = '1',
    stringData = 'test',
    schemaThing = {'iprcode': 4, 'cprcode': 123 , 'oprCode': '123'}
  ).save()
except Exception as e:
  print(e)


next(TestModel.query('1'))

None


{"schemaThing": {"iprcode": 4, "cprcode": 123, "oprCode": "123"}, "stringData": "test"}

### nested

In [ ]:
schemaUrl = 'https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/dev/order/order.yaml'
path = '/properties/schedule'
class TestModel(Model):
  class Meta:
    table_name="colab-test-sensitive-column"
    region = 'ap-southeast-1'
  id = UnicodeAttribute(hash_key=True)
  data = SchemaAttribute(schemaUrl = schemaUrl,path=path, null=True)
  def __repr__(self):
    return json.dumps(vars(self)['attribute_values'])

  
try:
  TestModel(
    id = '1',
    data = {'isExpress': 4, 'cprcode': 123 , 'oprCode': '123'}
  ).save()
except Exception as e:
  print('faulty data is rejected')
#   print(e)

try:
  TestModel(
    id = '1',
    data = {'isExpress': '4', 'cprcode': 123 , 'oprCode': '123'}
  ).save()
except Exception as e:
  print('valid data is rejected')
#   print(e)
  
next(TestModel.query('1'))  

faulty data is rejected
valid data is rejected


{}

### fail

In [ ]:
try:
  TestModel(
    phoneHash = '1',
    stringData = 'test',
    schemaThing = {'iprcode': '4', 'cprcode': 123 , 'oprCode': '123'}
  ).save()
except Exception as e:
  print(e)



next(TestModel.query('1'))

Attribute phoneHash specified does not exist


{}